In [ ]:
import pandas as pd
import numpy as np
from BeyondBlooms2024.config import name_dict, color_dict, color_month_dict,color_month_names 
from BeyondBlooms2024.config_file import (ABUNDANCES_FILE, CCMN_CON_MAP_PATH, CON_LOUVAIN_META_PATH,CON_LOUVAIN_NETWORK_PATH, ENRICH
,NUM_PERMUTATIONS, NUM_SAMPLES, NUM_CORES, METADATA_FILE, PRUNED_PVAL_CCMN_PATH,PVAL_CCMN_PATH,ENRICHED_META_PATH, RANDOM_PVAL_CCMN_PATH)

In [ ]:
df_enriched = pd.read_csv("tables/BeyondBlooms2024_Hellinger_True_14_Enriched_Hellinger_complete_network_table_meta_CON_CCM.csv")

In [ ]:
df_enriched

In [ ]:
df_enriched.dropna(subset="LouvainLabelD",inplace=True)
df_enriched["cluster_names"] = df_enriched["LouvainLabelD"].apply(lambda x: name_dict[str(int(x))])

In [ ]:
df_enriched["cluster_color"] = df_enriched["LouvainLabelD"].apply(lambda x: color_dict[str(int(x))])

In [ ]:
df_enriched.drop(columns=["Unnamed: 0"], inplace=True)
df_enriched = df_enriched[df_enriched["cluster_names"]!="RAUS"]

In [ ]:
df_abund = pd.read_csv(ABUNDANCES_FILE,sep=";")
df_abund.set_index("Unnamed: 0", inplace=True)
df_sqrt = np.sqrt(df_abund)
row_norms = np.linalg.norm(df_sqrt, axis=1)
df_normalized = df_sqrt.div(row_norms, axis=0)
df_abund = df_normalized
df_abund["Unnamed: 0"]=df_abund.index
df_abund["Unnamed: 0"] = pd.to_datetime(df_abund["Unnamed: 0"])
df_abund["month"]=df_abund["Unnamed: 0"].dt.month
df_abund["year"]=df_abund["Unnamed: 0"].dt.year
df_abund["year_month"]=df_abund["Unnamed: 0"].dt.strftime('%Y-%m')
df_abund.drop(columns="Unnamed: 0", inplace=True)
df_abund = df_abund.reset_index()
result_ym = df_abund.groupby("year_month").agg(sum).reset_index()
result_ym.set_index("year_month",inplace= True)
result_ym["year_month"] = result_ym.index
result_ym["year"] = result_ym.index
result_ym["year2"] = result_ym.index
result_ym["year"]=pd.to_datetime(result_ym["year"]).dt.year
result_ym["month"]=pd.to_datetime(result_ym["year2"]).dt.month
result = df_abund.groupby('month').agg(sum).reset_index()
result.set_index("month",inplace= True)
# Print the result
print(result.idxmax())
mm_dict = result.idxmax().to_dict()
df_enriched["MaxMonth"]=df_enriched["Nodes"].apply(lambda x: mm_dict[x])
df_enriched["MaxMonth"] = df_enriched["MaxMonth"].astype(str)
df_enriched["MaxMonthColor"]=df_enriched["MaxMonth"].map(color_month_dict)
df_enriched["MaxMonthName"]=df_enriched["MaxMonth"].map(color_month_names)

In [ ]:
df_enriched

In [ ]:
df_enriched.to_csv("tables/Enriched_Paper_Meta.csv",sep=",",index=False)